In [4]:
# import libraries
import pickle as pi
import sys
sys.path.append('../')

In [5]:
# import tensorboard
from torch.utils.tensorboard import SummaryWriter

In [6]:
# import the function class
from scripts.model import MolGen

In [8]:
# load data
data = []
with open('../ChEMBL_training/data/database_ChEMBL.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split("\n")[0])

#### GAN training on the ChEMBL dataset

In [10]:
# set the hyperparameters
lr = 1e-3
bs = 512
steps = 30000

In [11]:
# create model
gan_mol = MolGen(data, hidden_dim=128, lr=lr, device="cuda")

In [12]:
# create dataloader
loader = gan_mol.create_dataloader(data, batch_size=bs, shuffle=True, num_workers=10)

In [ ]:
# train model for 30000 steps
gan_mol.train_n_steps(loader, max_step=steps, evaluate_every=1000)

In [14]:
# save model
pi.dump(gan_mol, open('checkpoints/gan_mol.pkl', 'wb'))

### GAN fine-tuning on the coformer dataset

In [15]:
# load data
data = []
with open('data/database_CCDC.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split("\n")[0])

In [17]:
# load model
gan_mol = pi.load(open('checkpoints/gan_mol.pkl', 'rb'))

In [18]:
# set the hyperparameters
lr = 1e-3
bs = 256
steps = 1000

In [19]:
# set the new learning rate
gan_mol.lr = lr

In [20]:
# create dataloader
loader = gan_mol.create_dataloader(data, batch_size=bs, shuffle=True, num_workers=10)

In [ ]:
# train model for 1000 steps
gan_mol.train_n_steps(loader, max_step=steps, evaluate_every=100)

In [ ]:
# save model
pi.dump(gan_mol, open('checkpoints/gan_mol_ccdc_{0}_{1}_{2}k.pkl'.format(bs, '{:.0e}'.format(lr), steps//1000), 'wb'))